In [1]:
!nvcc --version
!nvidia-smi

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2021 NVIDIA Corporation
Built on Sun_Feb_14_21:12:58_PST_2021
Cuda compilation tools, release 11.2, V11.2.152
Build cuda_11.2.r11.2/compiler.29618528_0
Sun Jan  8 23:48:31 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   48C    P0    27W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |      

In [2]:
from google.colab import drive
drive.mount('/content/drive')
%cd '/content/drive/MyDrive/Colab Notebooks/DD2360 Applied GPU Programming KTH/Project'
!ls

Mounted at /content/drive
/content/drive/MyDrive/Colab Notebooks/DD2360 Applied GPU Programming KTH/Project
bin  datasets  include	Makefile  run_parallel_knn.ipynb  src  tmp.txt


In [44]:
!make clean
!make all
!chmod 755 ./bin/knn

rm -rf  src/terminal_user_input.o src/test.o src/knn.o
rm -rf bin/knn
rm -rf bin/test
[ -d src ] || mkdir src
nvcc -std=c++11 -I./include -O0 -g -G -Xcompiler -Wall -lm src/terminal_user_input.cpp -c -o src/terminal_user_input.o
nvcc -I./include -arch=sm_75 -std=c++11 -O0 -g -G -Xcompiler -Wall --compiler-bindir=g++ -lm src/test.cu -c -o src/test.o
nvcc -I./include -arch=sm_75 -std=c++11 -O0 -g -G -Xcompiler -Wall --compiler-bindir=g++ -lm src/terminal_user_input.o src/test.o -o bin/test
nvcc -I./include -arch=sm_75 -std=c++11 -O0 -g -G -Xcompiler -Wall --compiler-bindir=g++ -lm -DNDEBUG src/knn.cu -c -o src/knn.o
nvcc -I./include -arch=sm_75 -std=c++11 -O0 -g -G -Xcompiler -Wall --compiler-bindir=g++ -lm -DNDEBUG src/terminal_user_input.o src/knn.o -o bin/knn


**Debugging:**
- https://stackoverflow.com/questions/70626304/how-to-debug-a-cuda-google-colab-notebook
- https://stackoverflow.com/questions/14038589/what-is-the-canonical-way-to-check-for-errors-using-the-cuda-runtime-api
- https://www.olcf.ornl.gov/calendar/cuda-debugging/
- https://colab.research.google.com/drive/1GJOfTp56OeQRdE4u2_S7pUNRcJb4ik9X?usp=sharing
- https://docs.nvidia.com/cuda/cuda-gdb/index.htm
    - (cuda-gdb) break knn.cu:238
    - (cuda-gdb) break my_function
    - (cuda-gdb) next
    - (cuda-gdb) run
    - (cuda-gdb) print &array <br/>
      $1 = (@shared int (*)[0]) 0x20
    - (cuda-gdb) print array[0]@4 <br/>
      $2 = {0, 128, 64, 192}
    - (cuda-gdb) info cuda blocks
    - (cuda-gdb) info cuda threads
    - (cuda-gdb) up
    - (cuda-gdb) where

datasets/iris_dataset/iris.data.csv<br/>
datasets/iris_dataset/iris.data.small.csv

In [45]:
!./bin/knn
#!printf "set cuda memcheck on\nset cuda api_failures stop\ncatch throw\nr\nbt\ninfo locals\nthread 1\nbt\n" > ./tmp.txt
#!cuda-gdb -batch -x tmp.txt --args ./bin/knn
#!cuda-gdb ./bin/knn
#!compute-sanitizer --tool memcheck ./bin/knn

Filename: datasets/iris_dataset/iris.data.csv
k: 1, accuracy: 0.9597
k: 3, accuracy: 0.9597
k: 5, accuracy: 0.9664
k: 7, accuracy: 0.9664
k: 9, accuracy: 0.9664
k: 11, accuracy: 0.9799
k: 13, accuracy: 0.9664
k: 15, accuracy: 0.9732
k: 17, accuracy: 0.9732
k: 19, accuracy: 0.9799
k: 21, accuracy: 0.9732
k: 23, accuracy: 0.9664
k: 25, accuracy: 0.9664
k: 27, accuracy: 0.9664
k: 29, accuracy: 0.9530
k: 31, accuracy: 0.9463
k: 33, accuracy: 0.9530
k: 35, accuracy: 0.9530
k: 37, accuracy: 0.9463
k: 39, accuracy: 0.9463
k: 41, accuracy: 0.9463
k: 43, accuracy: 0.9396
k: 45, accuracy: 0.9463
k: 47, accuracy: 0.9463
k: 49, accuracy: 0.9396
k: 51, accuracy: 0.9195
k: 53, accuracy: 0.9329
k: 55, accuracy: 0.9329
k: 57, accuracy: 0.9329
k: 59, accuracy: 0.9329
k: 61, accuracy: 0.9262
k: 63, accuracy: 0.9262
k: 65, accuracy: 0.9128
k: 67, accuracy: 0.8993
k: 69, accuracy: 0.8993
k: 71, accuracy: 0.9060
k: 73, accuracy: 0.9060
k: 75, accuracy: 0.8792
k: 77, accuracy: 0.8658
k: 79, accuracy: 0.8725

In [46]:
!./bin/test


* Suite external_suite:
.........................Categories: Test Category
Categories: Class2
..
27 tests - 27 passed, 0 failed, 0 skipped (721889 ticks, 0.722 sec)

Total: 27 tests (721941 ticks, 0.722 sec), 31 assertions
Pass: 27, fail: 0, skip: 0.
